In [ ]:
!pip install openpyxl

In [ ]:
import re
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

## Arquivos remessa
<hr>

In [ ]:
encoding = 'iso-8859-1'

In [ ]:
lista_de_arquivos = ['dados/remessa (32).txt','dados/remessa (33).txt','dados/remessa (34).txt','dados/remessa (35).txt','dados/remessa (36).txt']

In [ ]:
for arquivo in lista_de_arquivos:
    with open(arquivo,'r', encoding = encoding) as file:
        conteudo = file.read()

    with open('remessa.txt','w', encoding = encoding) as arquivo_remessa:
        arquivo_remessa.write(conteudo)

with open('remessa.txt', 'r') as file:
            lines = file.readlines()

with open('remessa.txt', 'w') as file:
    for line in lines:
        # Substitui "DEFINITI" por 8 espaços em branco
        modified_line = line.replace("DEFINITI", " " * 8)
        file.write(modified_line)

In [ ]:
# Compile as expressões regulares
expressao_A = re.compile(r'[0-9][\s]{26}[0-9]{13}[\s]{354}[0-9]{6}')
expressao_B = re.compile(r'[0-9]{2}[A-Z]{7}[\s]{17}[0-9]{13}[\s]{7}[\s041A-Z]{41}[\s]{7}[0-9]{6}[\s]{294}[0-9]{6}')

# Inicialize uma lista para armazenar as linhas que não correspondem a nenhuma das expressões regulares
linhas_nao_correspondentes = []

with open("remessa.txt", "r") as arquivo_remessa:
    for linha in arquivo_remessa:
        if expressao_A.match(linha) or expressao_B.match(linha):
            linha_formatada = re.sub(r'\s{2,}', ',', linha.strip())
            # A linha corresponde a uma das expressões regulares, então não a adicionamos à lista
            continue
        else:
            # A linha não corresponde a nenhuma das expressões regulares, então podemos adicioná-la à lista
            linha_formatada = re.sub(r'\s{2,}', ',', linha.strip())
            linhas_nao_correspondentes.append(linha_formatada.strip())

# Crie um DataFrame com as linhas não correspondentes
df_linhas_nao_correspondentes = pd.DataFrame(linhas_nao_correspondentes, columns=["Linhas"])

df_remessa = df_linhas_nao_correspondentes['Linhas'].str.split(',', expand=True)

In [ ]:
df_remessa.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,0025000011080,5501083-JULHO/24,0010210054,10100102100540108240000000602682041,08N25072418,000000000000,0227829903000136A MAIS SUL HORTIFRUTI TRANSP. ...,AV. FERNANDO FERRARI,1001,1000090200041PORTO ALEGRE,RS,000002,None
1,1,0025000011080,5500403-JULHO/24,0010210135,10100102101350108240000001718337041,08N25072418,000000000000,0292222199000110A. BORGES TRANSP. E COM. DE BA...,AV. FERNANDO FERRARI,1001,1000090200041PORTO ALEGRE,RS,000003,None


In [ ]:
# Renomeando as colunas
new_column_names = {col: f'new_{col}' for col in df_remessa.columns}
df_remessa.rename(columns=new_column_names, inplace=True)

# Separando as colunas com os dados de Competência e Matrícula
df_remessa['COMPETENCIA'] = df_remessa['new_2'].apply(lambda x: x[8:] )
df_remessa['MATRICULA'] = df_remessa['new_2'].apply(lambda x: x[:7] )
df_remessa['VALOR DO TITULO REMESSA'] = df_remessa['new_4'].apply(lambda x : int(x[20:32])/100)
df_remessa['NOSSO NUM'] = df_remessa['new_3'].astype(int)
df_remessa['NOME DO SACADO/EMITENTE'] = df_remessa['new_7'].apply(lambda x: re.sub(r'[0-9]','', x))

In [ ]:
df_remessa[['COMPETENCIA','NOSSO NUM','MATRICULA','NOME DO SACADO/EMITENTE','VALOR DO TITULO REMESSA']].head()

,COMPETENCIA,NOSSO NUM,MATRICULA,NOME DO SACADO/EMITENTE,VALOR DO TITULO REMESSA
0,JULHO/24,10210054,5501083,A MAIS SUL HORTIFRUTI TRANSP. LTDA,6026.82
1,JULHO/24,10210135,5500403,A. BORGES TRANSP. E COM. DE BANANA,17183.37
2,JULHO/24,10210216,5500240,A. FLOR IMPORT. EXPORT. LTDA,10468.16
3,JULHO/24,10210399,5500980,ADMF HORTIFR. LTDA - ME,2581.70
4,JULHO/24,10210488,5500913,AGROATACADO COM. DE ALIMENTOS E PL,17600.51


## Arquivos de retorno
<hr>

In [ ]:
# Sua expressão regular para extrair os dados relevantes
expressao_A = (r"[0-9]{10}\s[0-9]{10}\s{4}[a-zA-Z0-9\s\W]{34}[0-9]{4}\s[0-9]{2}\/[0-9]{2}\/[0-9]{4}\s[0-9]{2"
              r"}\/[0-9]{2}\s{6}[A-Z]{8}\s{18}[A-Z][\s0-9.,]{19}(?:\s[A-Z])?$")

expressao_B = (r"[0-9]{10}\s[0-9]{10}\s{4}[a-zA-Z0-9\s\W]{34}[0-9]{4}\s[0-9]{2}\/[0-9]{2}\/[0-9]{4}\s[0-9]{2}\/[0-9]{2}\s{6}[A-Z]{8}\s{18}[A-Z][\s0-9.,]{19}(?:\s[A-Z])[\s]{4}[A-Z0-9]{2}[0-9.,\s]{17}[\s0-9A-Z]{3}")


# Lê o conteúdo do arquivo de texto
with open("dados/01.08.txt", "r") as retorno:
    texto_retorno = retorno.read()

# Extrai os dados relevantes usando a expressão regular
lista_A = re.findall(expressao_A, texto_retorno, re.MULTILINE)
lista_B = re.findall(expressao_B, texto_retorno, re.MULTILINE)

# Cria uma lista de listas
lista_bruta = []
for item in lista_A:
    # Verifica se o último valor está presente
    tipo_baixa = item[133].strip() if len(item) > 133 else "null"
    cod = item[135:140].strip() if len(item) > 140 else "null"
    deducao_acresc = item[140:157].strip() if len(item) > 157 else "null"
    ultimo_valor = item[159:160].strip() if len(item) > 159 else "null"

    lista_bruta.append([
        item[:10].strip(),
        item[11:21].strip(),
        item[25:57].strip(),
        item[57:63].strip(),
        item[63:74].strip(),
        item[75:85].strip(),
        item[86:94].strip(),
        item[112].strip(),
        item[113:132].strip(),
        tipo_baixa.strip(),
        cod.strip(),
        deducao_acresc.strip(),
        ultimo_valor.strip()
    ])
for item in lista_B:
    tipo_baixa = item[133].strip() if len(item) > 133 else "null"
    cod = item[135:140].strip() if len(item) > 140 else "null"
    deducao_acresc = item[140:157].strip() if len(item) > 157 else "null"
    ultimo_valor = item[159:160].strip() if len(item) > 159 else "null"

    lista_bruta.append([
        item[:10].strip(),
        item[11:21].strip(),
        item[25:57].strip(),
        item[57:63].strip(),
        item[63:74].strip(),
        item[75:85].strip(),
        item[86:94].strip(),
        item[112].strip(),
        item[113:132].strip(),
        tipo_baixa.strip(),
        cod.strip(),
        deducao_acresc.strip(),
        ultimo_valor.strip()
    ])

# Cria um DataFrame diretamente a partir da lista de listas
df_retorno = pd.DataFrame(lista_bruta)

In [ ]:
# Renomeando as colunas
new_column_names = {col: f'new_{col}' for col in df_retorno.columns}
df_retorno.rename(columns=new_column_names, inplace=True)

In [ ]:
df_retorno.rename(
    columns={
        'new_0':'NOSSO NUM',
        'new_1':'SEU NUMERO',
        'new_2':'NOME DO SACADO/EMITENTE',
        'new_3':'COBR',
        'new_4':'DT VENCTO',
        'new_5':'DT-OP',
        'new_6':'MOEDA',
        'new_7':'OP',
        'new_8':'VALOR DO TITULO',
        'new_9':'TB',
        'new_10':'COD',
        'new_11':'DEDUZ/ACRESCE',
        'new_12':'D/C'
    },
    inplace=True
)

# Formatando a coluna de valor do documento
df_retorno['VALOR DO TITULO'] = df_retorno['VALOR DO TITULO'].apply(lambda x: re.sub(r'\.', '', x))
df_retorno['VALOR DO TITULO'] = df_retorno['VALOR DO TITULO'].apply(lambda x: re.sub(r'\,', '.', x))
df_retorno['VALOR DO TITULO'] = df_retorno['VALOR DO TITULO'].apply(lambda x: re.sub(r'null', '0', x))
df_retorno['VALOR DO TITULO'] = df_retorno['VALOR DO TITULO'].astype(float)
# Formatando a coluna de valor dos acrescimos / descontos
df_retorno['DEDUZ/ACRESCE'] = df_retorno['DEDUZ/ACRESCE'].apply(lambda x: re.sub(r'\.', '', x))
df_retorno['DEDUZ/ACRESCE'] = df_retorno['DEDUZ/ACRESCE'].apply(lambda x: re.sub(r'\,', '.', x))
df_retorno['DEDUZ/ACRESCE'] = df_retorno['DEDUZ/ACRESCE'].apply(lambda x: re.sub(r'null', '0', x))
df_retorno['DEDUZ/ACRESCE'] = df_retorno['DEDUZ/ACRESCE'].astype(float)

# Coluna de "Nosso Número" & "Seu Número"
df_retorno[['NOSSO NUM','SEU NUMERO']] = df_retorno[['NOSSO NUM','SEU NUMERO']].astype(int)
df_retorno['TB'] = df_retorno['TB'].apply(lambda x:re.sub(r'null', '', x))
df_retorno['COD'] = df_retorno['COD'].apply(lambda x:re.sub(r'null', '', x))
df_retorno['D/C'] = df_retorno['D/C'].apply(lambda x:re.sub(r'null', '', x))

In [ ]:
df_retorno

,NOSSO NUM,SEU NUMERO,NOME DO SACADO/EMITENTE,COBR,DT VENCTO,DT-OP,MOEDA,OP,VALOR DO TITULO,TB,COD,DEDUZ/ACRESCE,D/C
0,10210135,10210135,A. BORGES TRANSP. E COM. DE BANA,0001,01/08/2024,01/08,DINHEIRO,C,17183.37,C,,0.0,
1,10210216,10210216,A. FLOR IMPORT. EXPORT. LTDA,0001,01/08/2024,01/08,DINHEIRO,C,10468.16,C,,0.0,
2,10161614,10161614,ADEMAR MARCOS FLACH,0104,01/08/2024,01/08,DINHEIRO,C,791.26,C,,0.0,
3,10161797,10161797,ADEMIR ILSON VOGT KLAGENBERG,0748,01/08/2024,01/08,DINHEIRO,C,755.99,C,,0.0,
4,10161878,10161878,ADEMIR MUSSOI,0748,01/08/2024,01/08,DINHEIRO,C,755.99,C,,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,10240905,10240905,W BECKER DA ROCHA,0633,01/08/2024,01/08,DINHEIRO,C,5582.74,C,,0.0,
326,10241081,10241081,W G COMERCIO DE FRUTAS LTDA,0341,01/08/2024,01/08,DINHEIRO,C,20586.17,C,,0.0,
327,10241243,10241243,WICKBOLD & NOSSO PAO LTDA,0341,01/08/2024,01/08,DINHEIRO,C,7278.30,C,,0.0,
328,10241324,10241324,WINTER COM. DE ALIM. E TRANSP. L,0010,01/08/2024,01/08,DINHEIRO,C,13453.86,,,0.0,


## Validação dos Pagamentos
<hr>

In [ ]:
df_boletos_pagos = pd.merge(
    left=df_remessa[['COMPETENCIA','NOSSO NUM','MATRICULA','NOME DO SACADO/EMITENTE','VALOR DO TITULO REMESSA']],
    right=df_retorno[['NOSSO NUM','COBR','DT VENCTO','DT-OP','VALOR DO TITULO','DEDUZ/ACRESCE']],
    on='NOSSO NUM',
    how='inner'
)
df_boletos_pagos = df_boletos_pagos[[
    'COMPETENCIA', 'DT-OP','DT VENCTO','MATRICULA','NOME DO SACADO/EMITENTE','NOSSO NUM','COBR',
    'VALOR DO TITULO REMESSA','VALOR DO TITULO','DEDUZ/ACRESCE'
]]

Boletos Pagos

In [ ]:
df_boletos_pagos

,COMPETENCIA,DT-OP,DT VENCTO,MATRICULA,NOME DO SACADO/EMITENTE,NOSSO NUM,COBR,VALOR DO TITULO REMESSA,VALOR DO TITULO,DEDUZ/ACRESCE
0,JULHO/24,01/08,01/08/2024,5500403,A. BORGES TRANSP. E COM. DE BANANA,10210135,0001,17183.37,17183.37,0.0
1,JULHO/24,01/08,01/08/2024,5500240,A. FLOR IMPORT. EXPORT. LTDA,10210216,0001,10468.16,10468.16,0.0
2,JULHO/24,01/08,01/08/2024,5500991,ALCIDES SOTER DE OLIVERA - EPP,10210542,0010,19863.22,19863.22,0.0
3,JULHO/24,01/08,01/08/2024,5500706,ALHO SUL LTDA,10210623,0010,17149.77,17149.77,0.0
4,JULHO/24,01/08,01/08/2024,5501062,AMD COM. DE HORTIFRUTI LTDA,10210968,0748,9577.41,9577.41,0.0
...,...,...,...,...,...,...,...,...,...,...
192,JULHO/24,01/08,01/08/2024,5500728,VOLNEI LUIZ MAGGIONI - ME,10240824,0033,21603.67,21603.67,0.0
193,JULHO/24,01/08,01/08/2024,5501092,W BECKER DA ROCHA,10240905,0633,5582.74,5582.74,0.0
194,JULHO/24,01/08,01/08/2024,5501088,W G COMERCIO DE FRUTAS LTDA,10241081,0341,20586.17,20586.17,0.0
195,JULHO/24,01/08,01/08/2024,5300656,WICKBOLD & NOSSO PAO LTDA,10241243,0341,7278.30,7278.30,0.0


Boletos não pagos

In [ ]:
nao_pagos = df_remessa.merge(df_boletos_pagos, on='NOSSO NUM', how='left', indicator=True)

# Filtre as linhas que estão apenas no df1
nao_pagos = nao_pagos[nao_pagos['_merge'] == 'left_only']
nao_pagos = nao_pagos[['COMPETENCIA_x','MATRICULA_x','NOSSO NUM','NOME DO SACADO/EMITENTE_x','VALOR DO TITULO REMESSA_x']]

In [ ]:
# Crie um novo Workbook
wb = Workbook()

# Remova a aba padrão criada automaticamente
wb.remove(wb.active)

# Adicione uma nova aba e escreva o primeiro DataFrame nela
ws1 = wb.create_sheet(title="Boletos Pagos")
for r in dataframe_to_rows(df_boletos_pagos, index=False, header=True):
    ws1.append(r)

# Adicione outra aba e escreva o segundo DataFrame nela
ws2 = wb.create_sheet(title="Não Pagos")
for r in dataframe_to_rows(nao_pagos, index=False, header=True):
    ws2.append(r)

# Salve o Workbook em um arquivo Excel
wb.save("Conciliação Bancária.xlsx")